### Image Preprocessing

In this section, we will use Tensorflow and Pillow (a wrapper package built on top of PIL, the Python Imaging Library), to obtain image data from files, resize images, and transform them. 

In [ ]:
import tensorflow as tf
import pillow
from pillow import PIL

### Properties of Image Data

Image data is made up of pixels, which are representations of the byte data used to depict the image. Each pixel consists of 8 bits (1 byte) for b/w images or 24 bits (3 bytes) for color images. 8 bits = 256 tonal levels = 0 - 255. 3 bytes are needed for colors: red, blue, green. 

You can import image data in Tensorflow using tf.read_file. 

tf.read_file can take in a file name and return the contents of the file as a tensor of type tf.string

When the input is an image, tf.read_file returns the raw byte data of the image file. Even though the raw byte data represents the image's pixels, it's not directly usable in this way. 

In [11]:
"""
def decode_image(filename, image_type, resize_shape, channels = 0):
    value = tf.read_file(filename)
"""

'\ndef decode_image(filename, image_type, resize_shape, channels = 0):\n    value = tf.read_file(filename)\n'

### Image Types

From the previous section, we can see that we can convert the file into usable pixel data. In Tensorflow, we have a few functions that can transform the raw data into pixel data:

• tf.image.decode_png - for PNG
• tf.iamge.decode_jpeg - for JPEG
• tf.image.decode_image - for decoding any image


We can use the PNG and JPEG - specific functions for code clarity. 

Moreover, tf.image.decode_image can decode GIFs, which have animation, so the return shape of tf.image.decode_image is (num_frames, height, width, channels). Because of this, it can be difficult to resize with tf.image.resize_images

We can specify the number of channels in the image. The function will try to figure it out based on the input, but we can also specify it. Channels = 0 (default) means the function will try to decode it. Channels = 1 specifies a grayscale image, channels = 3 specifies a color image, and channels 4 = can be used to specify a PNG image with an extra parameter of opacity. 

In [12]:
"""
# read in the image
value = tf.compat.v1.read_file('cat.jpg')
with tf.compat.v1.Session() as sess:
    # decode jpeg
    arr = sess.run(tf.io.decode_image(value, channels=1))
    print(arr.shape)
    print(repr(arr))
    
"""

"\n# read in the image\nvalue = tf.compat.v1.read_file('cat.jpg')\nwith tf.compat.v1.Session() as sess:\n    # decode jpeg\n    arr = sess.run(tf.io.decode_image(value, channels=1))\n    print(arr.shape)\n    print(repr(arr))\n    \n"

In [13]:
"""
# Decode image data from a file in Tensorflow
def decode_image(filename, image_type, resize_shape, channels=0):
    value = tf.read_file(filename)
    if(image_type == 'png'):
        decoded_image = tf.image.decode_png(value, channels = channels)
    elif(image_type == 'jpeg'):
        decoded_image = tf.image.decode_jpeg(value, channels = channels)
    else:
        decoded_image = tf.image.decode_image(value, channels = channels)
"""

"\n# Decode image data from a file in Tensorflow\ndef decode_image(filename, image_type, resize_shape, channels=0):\n    value = tf.read_file(filename)\n    if(image_type == 'png'):\n        decoded_image = tf.image.decode_png(value, channels = channels)\n    elif(image_type == 'jpeg'):\n        decoded_image = tf.image.decode_jpeg(value, channels = channels)\n    else:\n        decoded_image = tf.image.decode_image(value, channels = channels)\n"

### Image Scaling

To resize images in Tensorflow, we use tf.images.resize_images

It takes two required arguments: (1) the decoded image data and (2) the new size of the image (as a tuple of the new height and width)

The function below can expand or compress an image, then returns the pixel data of the resized image, with the same number of channels. If the resized dimensions don't match the aspect ratio (ratio of width to height) of the original image, you can get distortions.

Shrinking an image isn't hard. However, to expand an image and make it larger requires interpolating algorithms. The function has a keyword, 'method', that lets you specify the algorithm (four values possible: tf.image.ResizeMethod.BILINEAR, tf.image.ResizeMethod.NEAREST_NEIGHBOR, tf.image.ResizeMethod.BICUBIC, tf.image.ResizeMethod.AREA). The first three are useful for upsampling, or taking a smaller image and making it have more pixels (through some interpolation techniques). The last one is useful for downsampling. 

If we use tf.image.decode_image, we can't use tf.image.resize_images because the input for tf.image.decode_image needs to have a known number of dimensions, and tf.image.decode_image can have 3 or 4 dimensions. 

If we still need to crop an image with unknown size, we can use tf.image.resize_image_with_crop_or_pad, which resizes the pixel data by either padding the data with 0s (to increase the size) or by cropping the pixel data (to decrease the size). The output of this function is the same as the original data, since none of the individual pixels are transformed. To decrease the width and increase the height, cropping is applied along the width dimension, while padding is applied along the height dimension.

In [ ]:
"""
with tf.Session() as sess:
    print('Original: {}'.format(
        repr(sess.run(decoded_image))))  # Decoded image data
    resized_img = tf.image.resize_images(decoded_image, (3, 2))
    print('Resized: {}'.format(
        repr(sess.run(resized_img))))
"""

"""
with tf.Session() as sess:
    print('Original: {}'.format(
        repr(sess.run(decoded_image))))  # Decoded image data
    resized_img = tf.image.resize_image_with_crop_or_pad(
        decoded_image, 5, 2)
    print('Resized: {}'.format(
        repr(sess.run(resized_img))))
"""

"""
# Decode image data from a file in Tensorflow
def decode_image(filename, image_type, resize_shape, channels=0):
    value = tf.read_file(filename)
    if image_type == 'png':
        decoded_image = tf.image.decode_png(value, channels=channels)
    elif image_type == 'jpeg':
        decoded_image = tf.image.decode_jpeg(value, channels=channels)
    else:
        decoded_image = tf.image.decode_image(value, channels=channels)
    
    if(resize_shape and (image_type == 'png' or image_type == 'jpeg')):
        decoded_image = tf.image.resize_images(decoded_image, resize_shape)
    return decoded_image
"""

### Image Datasets

We can combine multiple images into one image dataset. To do so, we can use a Tensorflow dataset (tf.data.Dataset) to store all the images. We can create this dataset using the from_tensor_slices function. 

After creating a dataset, we need to decode each of the file's contents into usable pixel contents. Since we need to loop through each of the pictures, we will need to use the dataset's .map() function, and apply the decoding to each. Using .map() is more efficient than a for loop because the operations are done in parallel. 

In [ ]:
"""
image_paths = ['img1.jpg', 'img2.jpg']
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
def _map_fn(filename):
    # FUNCTION FROM PREVIOUS CHAPTERS
    return decode_image(...)
map_dataset = dataset.map(_map_fn)
"""

"""
import tensorflow as tf

def decode_image(filename, image_type, resize_shape, channels=0):
    value = tf.read_file(filename)
    if image_type == 'png':
        decoded_image = tf.image.decode_png(value, channels=channels)
    elif image_type == 'jpeg':
        decoded_image = tf.image.decode_jpeg(value, channels=channels)
    else:
        decoded_image = tf.image.decode_image(value, channels=channels)
    if resize_shape is not None and image_type in ['png', 'jpeg']:
        decoded_image = tf.image.resize_images(decoded_image, resize_shape)
    return decoded_image

# Return a dataset created from the image file paths
def get_dataset(image_paths, image_type, resize_shape, channels):
    # create tensor object to store image paths
    filename_tensor = tf.constant(image_paths)
    dataset = tf.data.Dataset.from_tensor_slices(filename_tensor)
    def _map_fn(filename):
        return decode_image(filename, image_type, resize_shape, channels = channels)
    return dataset.map(_map_fn)
"""

### Iterators

The way we extract image data from our dataset is through a tf.data.Iterator. This allows us to iterate through each of our images. We can use the get_next function to get the next element tensor. Note that the next-element tensor doesn't have an actual value until we execute the iteration process using tf.Session.

In [15]:
"""
# Get the decoded image data from the input image file paths
def get_image_data(image_paths, image_type=None, resize_shape=None, channels=0):
    # uses an Iterator object to get decoded image data from dataset
    dataset = get_dataset(image_paths, image_type, resize_shape, channels)
    # make iterator for dataset
    iterator = dataset.make_one_shot_iterator()
    # set up next element tensor for extracting data from dataset
    next_image = iterator.get_next()
"""

'\n# Get the decoded image data from the input image file paths\ndef get_image_data(image_paths, image_type=None, resize_shape=None, channels=0):\n    # uses an Iterator object to get decoded image data from dataset\n    dataset = get_dataset(image_paths, image_type, resize_shape, channels)\n    # make iterator for dataset\n    iterator = dataset.make_one_shot_iterator()\n    # set up next element tensor for extracting data from dataset\n    next_image = iterator.get_next()\n'

### Execution

When we iterate through the data, we can perform functions on each image. We execute the data extraction step using sess.run()

Each time we perform sess.run(next_image), we are iterating a single step through the dataset. So the first time we use sess.run(next_image), it’ll return the first pixel array in dataset (as a NumPy array), and the $i^{th}$ time it’ll return the $i^{th}$ pixel array in dataset

We can save the decoded images (which are returned as Numpy pixel data), using np.save, and then re-loaded using np.load.

In [ ]:
"""
it = dataset.make_one_shot_iterator()
next_image = it.get_next()
sess = tf.Session()
for i in range(3):
    sess.run(next_image)
"""

"""
arr = np.array([1, 2, 3])
# Saves to 'arr.npy'
np.save('arr.npy', arr)
# Also saves to 'arr.npy'
np.save('arr', arr)
# Loading from that file
arr_copy = np.load('arr.npy')
print(repr(arr_copy))
"""

"""
import tensorflow as tf

# Get the decoded image data from the input image file paths
def get_image_data(image_paths, image_type=None, resize_shape=None, channels=0):
    dataset = get_dataset(image_paths, image_type, resize_shape, channels)
    iterator = dataset.make_one_shot_iterator()
    next_image = iterator.get_next()
    
    # iniitlize list to return image pixel data
    image_data_list = []

    with tf.Session() as sess:
        # loop through images
        for i in range(len(image_paths)):
            # execute a single data extraction, append extracted pixel array to list
            image_data = sess.run(next_image)
            image_data_list.append(image_data)
    
    return image_data_list
"""

### Python Imaging Library (PIL)

The PIL module, maintained in the `pillow` package, allows us to do large scale image processing more conveniently than in Tensorflow. The PIL module has a submodule, `Image`, which is the main module used for image processing. We can create an `Image` object from an image file and obtain the image's pixel data. In addition to the resizing methods available to images in Tensorflow, `PIL` also permits filtering, through the `ImageFilter` submodule (e.g., blurring, image sharpening, etc.)

Below, we implement the pil_resize_image function

In [ ]:
"""
import numpy as np
from PIL import Image, ImageFilter

# Load and resize an image using PIL, and return its pixel data
def pil_resize_image(image_path, resize_shape,
    image_mode='RGBA', image_filter=None):
    # load image
    im = Image.open(image_path)
    # convert to specified image mode
    converted_im = im.convert(image_mode)
    # resize using Lanczos filtering
    resized_im = converted_im.resize(resize_shape, Image.LANCZOS)
    # check to see if an image filter is specified. If so, apply
    if image_filter:
        resized_im = resized_im.filter(image_filter)
    # return image pixel data, converted to a numpy array
    im_data = resized_im.getdata()
    return np.asarray(im_data)
"""